In [34]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.impute import SimpleImputer


In [21]:
random_state = 0

In [2]:
hfa_file_path = r'C:\\Users\\Penelope\\Documents\\ASSIGNMENT\\Health Facility Assessment strip.xlsx'


In [3]:
hfa = pd.read_excel (hfa_file_path)

In [5]:
hfa.head(6)

,Unnamed: 0,Unnamed: 0.1,number,form.health_centre_information.facility_name,form.facility_gps,form.health_centre_information.location_information.region_province,form.health_centre_information.location_information.district,form.health_centre_information.location_information.chiefdom,form.health_centre_information.location_information.facility_location,form.health_centre_information.facility_type,...,form.grp_infection_prevention_and_control.ql_water_sanitation_and_hygiene.score_infection_prevention_and_control_water_sanitation_and_hygiene,form.grp_infection_prevention_and_control.ql_water_sanitation_and_hygiene.score_max_infection_prevention_and_control_water_sanitation_and_hygiene,form.grp_infection_prevention_and_control.ql_disinfection_and_sterilization.score_infection_prevention_and_control_disinfection_and_sterilization,form.grp_infection_prevention_and_control.ql_disinfection_and_sterilization.score_max_infection_prevention_and_control_disinfection_and_sterilization,form.grp_infection_prevention_and_control.score_infection_prevention_and_control,form.grp_infection_prevention_and_control.score_max_infection_prevention_and_control,form.question1.score_logistics_patient_and_sample_transfer,form.question1.score_max_logistics_patient_and_sample_transfer,form.score_total,form.score_max_total
0,0,0,0,William hospital,"60.234598, -90.157892",Southern,Pujehun,New York,Jasonfort,community_health_post,...,4.0,5.0,1.0,5.0,10.5,27.0,19.5,44.0,48.5,102.0
1,1,1,1,Kathryn Hospital/CHC,"21.2454855, -107.509120",Southern,Bo,Arizona,West Jamesshire,community_health_post,...,1.5,5.0,0.0,5.0,5.5,27.0,21.0,44.0,35.5,102.0
2,2,2,2,Jessica Hospital,"-0.178657, -9.190029",Southern,Bo,New Mexico,Robertsside,community_health_post,...,2.0,5.0,0.0,5.0,10.0,27.0,33.5,44.0,55.5,102.0
3,3,3,3,Lauren CHC,"35.856989, -87.512790",Southern,Bonthe,Missouri,Port Christopher,mch_post,...,4.0,5.0,2.0,5.0,12.0,27.0,34.5,44.0,57.5,102.0
4,4,4,4,Tammy CHC,"54.7148835, 150.744813",Southern,Bonthe,Virginia,Reneeton,community_health_post,...,4.0,5.0,3.0,5.0,12.0,27.0,29.5,44.0,55.0,102.0
5,5,5,5,NaN,"22.8567535, 19.262410",NaN,NaN,NaN,NaN,NaN,...,0.0,5.0,0.0,5.0,0.0,27.0,NaN,NaN,NaN,NaN


In [10]:
#missing values
missing_values = hfa.isnull().sum()
print(missing_values.sum(), "missing")

total_cells = np.product(hfa.shape)
print(total_cells, "total")

print(missing_values.sum()/total_cells*100)

5674 missing
29870 total
18.995647807164378


In [11]:
#awesome
col_list = hfa.columns.to_list()
col_list[-2]

'form.score_total'

In [13]:
#select target
y = hfa["form.score_total"]

In [15]:
#just checking it worked
#print(y)

In [16]:
hfa_predictors = hfa.drop(['form.score_total'], axis=1)

In [25]:
#################################################################################
#NEED TO ENCODE CATEGORICAL AND NOT EXCLUDE DATA

In [17]:
#exclude non numerical data 
X = hfa_predictors.select_dtypes(exclude=['object'])

In [19]:
#print(X)

In [20]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2,
                                                      random_state=0)

In [23]:
#print(X_train)

In [26]:
# Function for comparing different approaches
def score_dataset(X_train, X_valid, y_train, y_valid):
    model = RandomForestRegressor(n_estimators=10, random_state=0)
    model.fit(X_train, y_train)
    preds = model.predict(X_valid)
    return mean_absolute_error(y_valid, preds)

In [27]:
# Get names of columns with missing values
cols_with_missing = [col for col in X_train.columns
                     if X_train[col].isnull().any()]

In [28]:
print(cols_with_missing)

['form.health_centre_information.facility_type_other', 'form.health_centre_information.capacity.number_consultation_rooms', 'form.health_centre_information.capacity.number_inpatient_beds', 'form.health_centre_information.capacity.number_maternity_beds', 'form.health_centre_information.group_number_employed.list_community_health_officer.cho_number_in_post', 'form.health_centre_information.group_number_employed.list_community_health_officer.cho_number_present_at_visit', 'form.health_centre_information.group_number_employed.list_community_health_technician.cht_number_in_post', 'form.health_centre_information.group_number_employed.list_community_health_technician.cht_number_present_at_visit', 'form.health_centre_information.group_number_employed.list_community_health_assistant.cha_number_in_post', 'form.health_centre_information.group_number_employed.list_community_health_assistant.cha_number_present_at_visit', 'form.health_centre_information.group_number_employed.list_community_midwives.c

In [29]:
# Get names of columns with problem values
cols_with_problems = [col for col in X_train.columns
                     if X_train[col].dtype=="object"]

In [30]:
print(cols_with_problems)

[]


In [ ]:
#check hfa col dtype

In [31]:
# Drop columns in training and validation data
reduced_X_train = X_train.drop(cols_with_missing, axis=1)
reduced_X_valid = X_valid.drop(cols_with_missing, axis=1)



In [36]:
print(reduced_X_train.shape)
print(reduced_X_valid.shape)

(116, 3)
(29, 3)


In [32]:
print("MAE from Approach 1 (Drop columns with missing values):")
print(score_dataset(reduced_X_train, reduced_X_valid, y_train, y_valid))

MAE from Approach 1 (Drop columns with missing values):


ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [37]:
# Imputation
my_imputer = SimpleImputer()
imputed_X_train = pd.DataFrame(my_imputer.fit_transform(X_train))
imputed_X_valid = pd.DataFrame(my_imputer.transform(X_valid))

# Imputation removed column names; put them back
imputed_X_train.columns = X_train.columns
imputed_X_valid.columns = X_valid.columns

print("MAE from Approach 2 (Imputation):")
print(score_dataset(imputed_X_train, imputed_X_valid, y_train, y_valid))

ValueError: Length mismatch: Expected axis has 71 elements, new values have 90 elements

In [56]:
# Make copy to avoid changing original data (when imputing)
X_train_plus = X_train.copy()
X_valid_plus = X_valid.copy()

# Make new columns indicating what will be imputed
for col in cols_with_missing:
    X_train_plus[col + '_was_missing'] = X_train_plus[col].isnull()
    X_valid_plus[col + '_was_missing'] = X_valid_plus[col].isnull()

# Imputation
my_imputer = SimpleImputer()
imputed_X_train_plus = pd.DataFrame(my_imputer.fit_transform(X_train_plus))
imputed_X_valid_plus = pd.DataFrame(my_imputer.transform(X_valid_plus))

# Imputation removed column names; put them back
imputed_X_train_plus.columns = X_train_plus.columns
imputed_X_valid_plus.columns = X_valid_plus.columns

print("MAE from Approach 3 (An Extension to Imputation):")
print(score_dataset(imputed_X_train_plus, imputed_X_valid_plus, y_train, y_valid))


MAE from Approach 3 (An Extension to Imputation):
7.701330532212886
